In [44]:
import numpy as np
import math
from PIL import Image
from skimage.color import rgb2gray
from sklearn import svm
import pandas as pd
import os
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from imblearn.over_sampling import SMOTE

## PART 1

##### Helper functions for grouping pixels

In [30]:
def group_pixels(img, bin_number):
    max_width = np.max(img) - np.min(img)
    bins = []
    for i in range(1, bin_number):
        bins.append(math.floor(i * (max_width / bin_number)))
    bins.append(np.max(img) + 1)
    return bins

In [31]:
def find_level(pix, bins):
    group_num = 0
    for binn in bins:
        if binn > pix:
            return group_num
        group_num += 1

#### Functions for Calculation of Cooccurence Matrix and Haralick features 

In [32]:
def calculateCooccurenceMatrix(grayImg, binNumber, di, dj):
    refined = grayImg * 256
    bins = group_pixels(refined, binNumber)
    co_occur_mtx = np.zeros((binNumber, binNumber))
    
    for i in range(len(refined)):
        new_i = i + di
        if new_i < 0 or new_i >= len(refined):
            continue
        for j in range(len(refined[0])):
            new_j = j + dj
            if new_j < 0 or new_j >= len(refined[0]):
                continue
            initial = find_level(refined[i, j], bins)
            end = find_level(refined[new_i, new_j], bins)
            co_occur_mtx[initial, end] += 1
    
    return co_occur_mtx

In [33]:
def calculateAccumulatedCooccurenceMatrix(grayImg, binNumber, d):
    connectivity_list = [(d, 0), (d, d), (0, d), (-d, d), (-d, 0), (-d, -d), (0, -d), (d, -d)]
    accM = np.zeros((binNumber, binNumber))
    for elem in connectivity_list:
        accM += calculateCooccurenceMatrix(grayImg, binNumber, elem[0], elem[1])
    return accM

In [34]:
def calculateCooccurrenceFeatures(accM):
    ##noramlization
    normalized = accM/accM.sum()
    
    ang_sc_mom = np.square(normalized).sum()
    max_prob = np.max(normalized)
    
    mean_i = np.sum(normalized, axis = 0).mean()
    mean_j = np.sum(normalized, axis = 1).mean()
    std_i = np.sum(normalized, axis = 0).std()
    std_j = np.sum(normalized, axis = 1).std()
    
    inv_diff_mom = 0
    contrast = 0
    entropy = 0
    corr = 0
    
    for i in range(len(accM)):
        for j in range(len(accM)):
            inv_diff_mom += normalized[i, j] / (1 + (i-j)*(i-j))
            contrast += (i-j)*(i-j)*normalized[i, j]
            if normalized[i, j] != 0:
                entropy -= normalized[i,j]*math.log(normalized[i,j])
            corr += i*j*normalized[i,j]
    
    corr = (corr - mean_i*mean_j)/(std_i*std_j)
    
    ## returned in a dictionary data structure to ease of readability
    return {'Angular Second Moment': ang_sc_mom,
            'Maximum Probability': max_prob,
            'Inverse Difference Moment': inv_diff_mom,
            'Contrast': contrast,
            'Entropy': entropy,
            'Correlation': corr}

## Part 2

#### In the below cell, haralick features for all training images are obtained (class informations and features are kept in a dataframe for ease of manupilation)

In [468]:
df = pd.DataFrame()
labels = np.loadtxt("./dataset/training_labels.txt")
##list for class-balancing issues
df['labels'] = labels
ans = []
for i in range(1,187):
    name = 'tr' + str(i) + '.jpg'
    image = Image.open("./dataset/training" + '/' + name)
    img_data = np.asarray(image)
    gray = rgb2gray(img_data)
    coor = calculateCooccurrenceFeatures(calculateAccumulatedCooccurenceMatrix(gray,8,10))
    ans.append(coor)

df['Angular Second Moment'] = [sub['Angular Second Moment'] for sub in ans]
df['Maximum Probability'] = [sub['Maximum Probability'] for sub in ans]
df['Inverse Difference Moment'] = [sub['Inverse Difference Moment'] for sub in ans]
df['Contrast'] = [sub['Contrast'] for sub in ans]
df['Entropy'] = [sub['Entropy'] for sub in ans]
df['Correlation'] = [sub['Correlation'] for sub in ans]
##normalization
for column in df.columns:
    if column == 'labels':
        continue
    df[column] = (df[column] - df[column].mean())/df[column].std()

#### Dataframe example for training data

In [ ]:
df

#### In this cell Synthetic Minority Oversampling Technique (SMOTE) is used to solve the imbalanced class problem
#### number instances for classes before [1.0: 60, 2.0: 88, 3.0: 38] -> [1.0: 88, 2.0: 88, 3.0: 88]

In [469]:
features = [col for col in df.columns if col != 'labels']
train_y = df['labels']
train_X = df[features]
sampler = SMOTE(sampling_strategy='auto')
X_over, y_over = sampler.fit_resample(train_X, train_y)

#### same process for test data as for training data

In [473]:
df2 = pd.DataFrame()
labels = np.loadtxt("./dataset/test_labels.txt")
##list for class-balancing issues
df2['labels'] = labels
ans = []
for i in range(1,145):
    name = 'ts' + str(i) + '.jpg'
    image = Image.open("./dataset/test" + '/' + name)
    img_data = np.asarray(image)
    gray = rgb2gray(img_data)
    coor = calculateCooccurrenceFeatures(calculateAccumulatedCooccurenceMatrix(gray,8,10))
    ans.append(coor)

df2['Angular Second Moment'] = [sub['Angular Second Moment'] for sub in ans]
df2['Maximum Probability'] = [sub['Maximum Probability'] for sub in ans]
df2['Inverse Difference Moment'] = [sub['Inverse Difference Moment'] for sub in ans]
df2['Contrast'] = [sub['Contrast'] for sub in ans]
df2['Entropy'] = [sub['Entropy'] for sub in ans]
df2['Correlation'] = [sub['Correlation'] for sub in ans]
##normalization
for column in df.columns:
    if column == 'labels':
        continue
    df2[column] = (df2[column] - df2[column].mean())/df2[column].std()

In [474]:
test_y = df2['labels']
test_X = df2[features]

#### in the below training process for SVM with linear kernel and results

In [692]:
## For tuning purposes 

#params = {'C': np.arange(100, 10010, 1000)}
# grid = GridSearchCV(estimator=linear_kernel, param_grid=params)
# grid.fit(X_over, y_over)
#print(grid.best_estimator_)


## Support vector machine with linear kernel training and results
linear_kernel = svm.SVC(C=1400, kernel='linear')
linear_kernel.fit(X_over, y_over)
linear_pred = linear_kernel.predict(X_over)
mat = metrics.confusion_matrix(y_over, linear_pred)
print(mat)
print(mat.diagonal()/mat.sum(axis=1))
print(metrics.classification_report(y_over, linear_pred))

[[71  8  9]
 [ 8 70 10]
 [ 5  6 77]]
[0.80681818 0.79545455 0.875     ]
              precision    recall  f1-score   support

         1.0       0.85      0.81      0.83        88
         2.0       0.83      0.80      0.81        88
         3.0       0.80      0.88      0.84        88

    accuracy                           0.83       264
   macro avg       0.83      0.83      0.83       264
weighted avg       0.83      0.83      0.83       264



#### Same process for Radial basis kernel

In [693]:
rbf_kernel = svm.SVC(C=7000, gamma=0.015, kernel='rbf')
rbf_kernel.fit(X_over, y_over)
rbf_pred = rbf_kernel.predict(X_over)
mat = metrics.confusion_matrix(y_over, rbf_pred)
print(mat.diagonal()/mat.sum(axis=1))
print(metrics.classification_report(y_over, rbf_pred))

[0.875      0.875      0.90909091]
              precision    recall  f1-score   support

         1.0       0.87      0.88      0.87        88
         2.0       0.89      0.88      0.88        88
         3.0       0.91      0.91      0.91        88

    accuracy                           0.89       264
   macro avg       0.89      0.89      0.89       264
weighted avg       0.89      0.89      0.89       264



#### Creation of contingency matrices for all classes and Mc nemar's test

In [694]:
contingency_1 = [[0,0], [0,0]]
contingency_2 = [[0,0], [0,0]]
contingency_3 = [[0,0], [0,0]]
for i in range(len(y_over)):
    if y_over[i] == 1.0:
        if y_over[i] == linear_pred[i] and y_over[i] == rbf_pred[i]:
            contingency_1[0][0] += 1
        elif y_over[i] == linear_pred[i] and y_over[i] != rbf_pred[i]:
            contingency_1[0][1] += 1
        elif y_over[i] != linear_pred[i] and y_over[i] == rbf_pred[i]:
            contingency_1[1][0] += 1
        else: 
            contingency_1[1][1] += 1
    elif y_over[i] == 2.0:
        if y_over[i] == linear_pred[i] and y_over[i] == rbf_pred[i]:
            contingency_2[0][0] += 1
        elif y_over[i] == linear_pred[i] and y_over[i] != rbf_pred[i]:
            contingency_2[0][1] += 1
        elif y_over[i] != linear_pred[i] and y_over[i] == rbf_pred[i]:
            contingency_2[1][0] += 1
        else: 
            contingency_2[1][1] += 1
    else:
        if y_over[i] == linear_pred[i] and y_over[i] == rbf_pred[i]:
            contingency_3[0][0] += 1
        elif y_over[i] == linear_pred[i] and y_over[i] != rbf_pred[i]:
            contingency_3[0][1] += 1
        elif y_over[i] != linear_pred[i] and y_over[i] == rbf_pred[i]:
            contingency_3[1][0] += 1
        else: 
            contingency_3[1][1] += 1
        
print(contingency_1)
print(contingency_2)
print(contingency_3)

[[69, 2], [8, 9]]
[[70, 0], [7, 11]]
[[71, 6], [9, 2]]


In [698]:
from statsmodels.stats.contingency_tables import mcnemar
print(mcnemar(contingency_2, exact=False))

pvalue      0.02334220201289086
statistic   5.142857142857143


## Part 3

#### Function to obtain Haralick features for grid-based approach

In [593]:
def grid_based_features(img, binNumber, d, N):
    lenx = len(img)
    leny = len(img[0])
    ans = []
    for i in range(N):
        for j in range(N):
            grid = img[int(lenx * i / N):int(lenx * (i + 1) / N), int(leny * j / N):int(leny * (j + 1) / N)]
            feature = calculateCooccurrenceFeatures(calculateAccumulatedCooccurenceMatrix(grid, binNumber, d))
            ans.append(feature)
    return {'Angular Second Moment': np.array([sub['Angular Second Moment'] for sub in ans]).mean(),
            'Maximum Probability': np.array([sub['Maximum Probability'] for sub in ans]).mean(),
            'Inverse Difference Moment': np.array([sub['Inverse Difference Moment'] for sub in ans]).mean(),
            'Contrast': np.array([sub['Contrast'] for sub in ans]).mean(),
            'Entropy': np.array([sub['Entropy'] for sub in ans]).mean(),
            'Correlation': np.array([sub['Correlation'] for sub in ans]).mean()}

#### Features with grid-based approach for training data

In [594]:
df3 = pd.DataFrame()
labels = np.loadtxt("./dataset/training_labels.txt")
##list for class-balancing issues
df3['labels'] = labels
ans = []
for i in range(1,187):
    name = 'tr' + str(i) + '.jpg'
    image = Image.open("./dataset/training" + '/' + name)
    img_data = np.asarray(image)
    gray = rgb2gray(img_data)a
    coor = grid_based_features(gray, 8, 10, 4)
    ans.append(coor)

df3['Angular Second Moment'] = [sub['Angular Second Moment'] for sub in ans]
df3['Maximum Probability'] = [sub['Maximum Probability'] for sub in ans]
df3['Inverse Difference Moment'] = [sub['Inverse Difference Moment'] for sub in ans]
df3['Contrast'] = [sub['Contrast'] for sub in ans]
df3['Entropy'] = [sub['Entropy'] for sub in ans]
df3['Correlation'] = [sub['Correlation'] for sub in ans]
##normalization
for column in df3.columns:
    if column == 'labels':
        continue
    df3[column] = (df3[column] - df3[column].mean())/df3[column].std()

#### Again oversampling

In [596]:
grid_train_y = df3['labels']
grid_train_X = df3[features]
sampler = SMOTE(sampling_strategy='auto')
X_over_grid, y_over_grid = sampler.fit_resample(grid_train_X, grid_train_y)

#### Feature extraction for test data

In [601]:
df4 = pd.DataFrame()
labels = np.loadtxt("./dataset/test_labels.txt")
##list for class-balancing issues
df4['labels'] = labels
ans = []
for i in range(1,145):
    name = 'ts' + str(i) + '.jpg'
    image = Image.open("./dataset/test" + '/' + name)
    img_data = np.asarray(image)
    gray = rgb2gray(img_data)
    coor = grid_based_features(gray, 8, 10, 4)
    ans.append(coor)

df4['Angular Second Moment'] = [sub['Angular Second Moment'] for sub in ans]
df4['Maximum Probability'] = [sub['Maximum Probability'] for sub in ans]
df4['Inverse Difference Moment'] = [sub['Inverse Difference Moment'] for sub in ans]
df4['Contrast'] = [sub['Contrast'] for sub in ans]
df4['Entropy'] = [sub['Entropy'] for sub in ans]
df4['Correlation'] = [sub['Correlation'] for sub in ans]
##normalization
for column in df4.columns:
    if column == 'labels':
        continue
    df4[column] = (df4[column] - df4[column].mean())/df4[column].std()

In [603]:
test_y_grid = df4['labels']
test_X_grid = df4[features]

#### Training SVM with linear kernel (grid-based approach, parameters obtained as a result of tuning)

In [676]:
linear_kernel_grid = svm.SVC(C=20, kernel='linear')
linear_kernel_grid.fit(X_over_grid, y_over_grid)
linear_pred_grid = linear_kernel_grid.predict(X_over_grid)
mat = metrics.confusion_matrix(y_over_grid, linear_pred_grid)
print(mat)
print(mat.diagonal()/mat.sum(axis=1))
print(metrics.classification_report(y_over_grid, linear_pred_grid))

[[69 12  7]
 [ 5 70 13]
 [16  4 68]]
[0.78409091 0.79545455 0.77272727]
              precision    recall  f1-score   support

         1.0       0.77      0.78      0.78        88
         2.0       0.81      0.80      0.80        88
         3.0       0.77      0.77      0.77        88

    accuracy                           0.78       264
   macro avg       0.78      0.78      0.78       264
weighted avg       0.78      0.78      0.78       264



#### SVM with RBF kernel

In [684]:
rbf_kernel_grid = svm.SVC(C=2300, gamma=0.007, kernel='rbf')
rbf_kernel_grid.fit(X_over_grid, y_over_grid)
rbf_pred_grid = rbf_kernel_grid.predict(X_over_grid)
mat = metrics.confusion_matrix(y_over_grid, rbf_pred_grid)
print(mat)
print(mat.diagonal()/mat.sum(axis=1))
print(metrics.classification_report(y_over_grid, rbf_pred_grid))

[[80  5  3]
 [ 8 74  6]
 [13  2 73]]
[0.90909091 0.84090909 0.82954545]
              precision    recall  f1-score   support

         1.0       0.79      0.91      0.85        88
         2.0       0.91      0.84      0.88        88
         3.0       0.89      0.83      0.86        88

    accuracy                           0.86       264
   macro avg       0.87      0.86      0.86       264
weighted avg       0.87      0.86      0.86       264



## Extension 1 (SIFT features)

In [9]:
import cv2

#### Helper function to crop image using the keypoints

In [35]:
def keypoint_features(img, binNumber, d, coor):
    lenx = len(img)
    leny = len(img[0])
    ans = []
    for i in coor:
        crop_x_left = np.round(i[0]) - 16
        crop_x_right = np.round(i[0]) + 16
        crop_y_up = np.round(i[1]) - 16
        crop_y_down = np.round(i[1]) + 16
        if crop_x_left < 0 or crop_x_right >= lenx or crop_y_up < 0 or crop_y_down >= leny:
            continue
        window = img[int(crop_x_left):int(crop_x_right), int(crop_y_up):int(crop_y_down)]
        feature = calculateCooccurrenceFeatures(calculateAccumulatedCooccurenceMatrix(window, binNumber, d))
        ans.append(feature)
    return {'Angular Second Moment': np.array([sub['Angular Second Moment'] for sub in ans]).mean(),
            'Maximum Probability': np.array([sub['Maximum Probability'] for sub in ans]).mean(),
            'Inverse Difference Moment': np.array([sub['Inverse Difference Moment'] for sub in ans]).mean(),
            'Contrast': np.array([sub['Contrast'] for sub in ans]).mean(),
            'Entropy': np.array([sub['Entropy'] for sub in ans]).mean(),
            'Correlation': np.array([sub['Correlation'] for sub in ans]).mean()}

#### Feature extraction for training data (SIFT keypoints are obtained and used)

In [36]:
df5 = pd.DataFrame()
sift = cv2.SIFT_create()
labels = np.loadtxt("./dataset/training_labels.txt")
##list for class-balancing issues
df5['labels'] = labels
ans = []
for i in range(1,187):
    print(i)
    name = 'tr' + str(i) + '.jpg'
    image = Image.open("./dataset/training" + '/' + name)
    img_data = np.asarray(image)
    gray = rgb2gray(img_data)
    keypoints, descriptors = sift.detectAndCompute(img_data, None)
    coor = [key.pt for key in keypoints]
    feat = keypoint_features(gray, 8, 10, coor)
    ans.append(feat)

df5['Angular Second Moment'] = [sub['Angular Second Moment'] for sub in ans]
df5['Maximum Probability'] = [sub['Maximum Probability'] for sub in ans]
df5['Inverse Difference Moment'] = [sub['Inverse Difference Moment'] for sub in ans]
df5['Contrast'] = [sub['Contrast'] for sub in ans]
df5['Entropy'] = [sub['Entropy'] for sub in ans]
df5['Correlation'] = [sub['Correlation'] for sub in ans]
##normalization
for column in df5.columns:
    if column == 'labels':
        continue
    df5[column] = (df5[column] - df5[column].mean())/df5[column].std()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186


In [61]:
df5

,labels,Angular Second Moment,Maximum Probability,Inverse Difference Moment,Contrast,Entropy,Correlation
0,1.0,1.093505,1.071023,1.044692,-0.894533,-1.055762,-0.846020
1,1.0,1.068121,1.086060,1.068913,-0.948292,-1.049326,-0.870583
2,1.0,0.743602,0.838123,0.817733,-0.767426,-0.786393,-0.781005
3,1.0,0.111899,0.046290,0.012772,0.009999,-0.046942,-0.031371
4,1.0,-0.651386,-0.698687,-0.749010,0.744767,0.712330,0.526621
...,...,...,...,...,...,...,...
181,3.0,-0.306867,-0.165079,-0.106896,-0.133058,0.130016,-0.180888
182,3.0,0.070991,0.170553,0.200525,-0.286816,-0.157317,-0.312175
183,3.0,-0.222961,-0.044212,0.006218,-0.198211,0.043873,-0.281362
184,3.0,0.083610,0.249245,0.267277,-0.379272,-0.223320,-0.472820


#### Oversampling again

In [41]:
features = [col for col in df5.columns if col != 'labels']
SIFT_train_y = df5['labels']
SIFT_train_X = df5[features]
sampler = SMOTE(sampling_strategy='auto')
X_over_SIFT, y_over_SIFT = sampler.fit_resample(SIFT_train_X, SIFT_train_y)

#### Same process for test data

In [38]:
df6 = pd.DataFrame()
labels = np.loadtxt("./dataset/test_labels.txt")
##list for class-balancing issues
df6['labels'] = labels
ans = []
for i in range(1,145):
    print(i)
    name = 'ts' + str(i) + '.jpg'
    image = Image.open("./dataset/test" + '/' + name)
    img_data = np.asarray(image)
    gray = rgb2gray(img_data)
    keypoints, descriptors = sift.detectAndCompute(img_data, None)
    coor = [key.pt for key in keypoints]
    feat = keypoint_features(gray, 8, 10, coor)
    ans.append(feat)

df6['Angular Second Moment'] = [sub['Angular Second Moment'] for sub in ans]
df6['Maximum Probability'] = [sub['Maximum Probability'] for sub in ans]
df6['Inverse Difference Moment'] = [sub['Inverse Difference Moment'] for sub in ans]
df6['Contrast'] = [sub['Contrast'] for sub in ans]
df6['Entropy'] = [sub['Entropy'] for sub in ans]
df6['Correlation'] = [sub['Correlation'] for sub in ans]
##normalization
for column in df6.columns:
    if column == 'labels':
        continue
    df6[column] = (df6[column] - df6[column].mean())/df6[column].std()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144


In [43]:
test_y_SIFT = df6['labels']
test_X_SIFT = df6[features]

#### SVM training with linear kernel (SIFT)

In [104]:
# best = 0
# best_c = 0
# for c in np.arange(3100, 4200, 100):
#     linear_kernel = svm.SVC(C=c, kernel='linear')
#     score = cross_val_score(linear_kernel, X_over_SIFT, y_over_SIFT, cv=10).mean()
#     if score > best:
#         best = score
#         best_c = c
        
# print(best, best_c)
linear_kernel_SIFT = svm.SVC(C=4100, kernel='linear')
linear_kernel_SIFT.fit(X_over_SIFT, y_over_SIFT)
linear_pred_SIFT = linear_kernel_SIFT.predict(test_X_SIFT)
mat = metrics.confusion_matrix(test_y_SIFT, linear_pred_SIFT)
print(mat)
print(mat.diagonal()/mat.sum(axis=1))
print(metrics.classification_report(test_y_SIFT, linear_pred_SIFT))

[[31  3 14]
 [ 1 51  5]
 [15  5 19]]
[0.64583333 0.89473684 0.48717949]
              precision    recall  f1-score   support

         1.0       0.66      0.65      0.65        48
         2.0       0.86      0.89      0.88        57
         3.0       0.50      0.49      0.49        39

    accuracy                           0.70       144
   macro avg       0.67      0.68      0.68       144
weighted avg       0.70      0.70      0.70       144



In [105]:
rbf_kernel_SIFT = svm.SVC(C=5000, gamma=0.007, kernel='rbf')
rbf_kernel_SIFT.fit(X_over_SIFT, y_over_SIFT)
rbf_pred_SIFT = rbf_kernel_SIFT.predict(test_X_SIFT)
mat = metrics.confusion_matrix(test_y_SIFT, rbf_pred_SIFT)
print(mat)
print(mat.diagonal()/mat.sum(axis=1))
print(metrics.classification_report(test_y_SIFT, rbf_pred_SIFT))

[[41  1  6]
 [ 2 51  4]
 [ 3  9 27]]
[0.85416667 0.89473684 0.69230769]
              precision    recall  f1-score   support

         1.0       0.89      0.85      0.87        48
         2.0       0.84      0.89      0.86        57
         3.0       0.73      0.69      0.71        39

    accuracy                           0.83       144
   macro avg       0.82      0.81      0.82       144
weighted avg       0.83      0.83      0.83       144

